In [1]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import math
from sklearn.feature_extraction import image
from sklearn.cluster import spectral_clustering
from moviepy.editor import VideoFileClip
import tqdm

In [2]:
def save(image,name):
    name=""+str(name)+".png"
    cv2.imwrite(name,image)


In [3]:
def grayen(image):
    grayenn = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return grayenn

In [4]:
def darken(image):
    new_image=np.zeros_like(image)
    alpha=0.19
    beta=-35
    new_image=np.clip(np.multiply(alpha,image)+beta,0,255)
    return new_image

In [5]:
def brighten(image):
    new_image=np.zeros_like(image)
    gamma=0.5
    new_image=np.clip(np.multiply(np.power(np.multiply(1/255,image),gamma),255),0,255)
    return new_image

In [6]:
def colour_threshing(image):
    image = np.where(image < 210, 0, 255)

    return image

In [7]:
def perspective(image):
    h,w=image.shape
    pts1 = np.float32([[h/3,0],[h/3,w-1],[h-1,0],[h-1,w-1]])
    pts2 = np.float32([[0,0],[300,0],[0,300],[300,300]])

    M = cv2.getPerspectiveTransform(pts1,pts2)

    dst = cv2.warpPerspective(median_image,M,(300,300))
    return dst

In [8]:
def roi_mask(image):
    mask=np.zeros_like(image)
    height,width = image.shape
    a3 = np.array( [[[0,200],[1500,200],[width-1,800],[width-1,height-1],[0,height-1]]], dtype=np.int32 )
    cv2.fillPoly(mask,a3,255)
    return mask

In [9]:
def white_pixels(image,x,y):
    h=window_h
    w=window_w
    sum1=np.sum(combo[y-h:y,x:x+w],axis=0).reshape((1,w))
    sum2=np.sum(sum1,axis=1)
    return sum2/255

In [10]:
def centroid(rect1x):
    he=rect1x.shape[0]
    wi=rect1x.shape[1]
    sum1x=0
    sum1y=0
    for i in range(he):
        for j in range(wi):
            sum1x=sum1x+j*rect1x[i,j]*(1/255)
            sum1y=sum1y+i*rect1x[i,j]*(1/255)
    sum1x=int(sum1x/window_w)
    sum1y=int(sum1y/window_h)
    return [sum1x,sum1y]

In [11]:
def yellow_black(image):
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    yellow_lo=np.array([25,175,175])
    yellow_hi=np.array([35,255,255])

    mask=cv2.inRange(hsv,yellow_lo,yellow_hi)
    
    img=np.copy(image)
    img[mask>0]=(0,0,0)

    return img

In [12]:
def green_white(image):
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    green_lo=np.array([20,10,30])
    green_hi=np.array([80,255,255])

    mask=cv2.inRange(hsv,green_lo,green_hi)
    img=np.copy(image)
    img[mask>0]=(255,255,255)

    return img

In [13]:
def binary(image):
    img=np.copy(image)
    img=np.where(img!=(255,255,255),(0,0,0),(255,255,255))
    return img

In [14]:
def vid_pipeline(img):
    x=0
    y=0
    width= img.shape[1]
    height=img.shape[0]
    crop_image = img[y:y+(int)(height*(5/6)), x:x+width]

    width= crop_image.shape[1]
    height=crop_image.shape[0]
    
    yellow=yellow_black(cv2.medianBlur(crop_image.astype(np.uint8),25))
    green=green_white(yellow)
    masked=binary(green)
    masked = cv2.medianBlur(masked.astype(np.uint8),9)
    
    crop_image2=np.copy(crop_image)
    crop_image2 = cv2.medianBlur(np.uint8(crop_image2.astype(np.float32)),25)
    
    median = cv2.medianBlur(np.uint8(crop_image2),35)
    
    edge=cv2.Canny(np.uint8(median),10,100)
    
    dilation = cv2.dilate(np.uint8(edge),np.ones((5,5)),iterations = 1)
    
    contours, hierarchy = cv2.findContours((np.uint8(dilation)), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    con=cv2.drawContours((np.uint8(edge)), contours, -1,255, 30)
    
    imcon=np.where(con==255,0,255)
    
    rgb=cv2.cvtColor(con,cv2.COLOR_GRAY2RGB)
    
    crop_image=np.where(rgb==(255,255,255),crop_image*0,crop_image)
    
    
#     
# 
    gray=grayen(crop_image2)
    brighten_image=brighten(gray)
    bw_image=colour_threshing(brighten_image)
    median_image = cv2.medianBlur(np.uint8(bw_image.astype(np.float32)),15)
    mask=roi_mask(median_image)
    combo=cv2.bitwise_and(mask,median_image)
    # save(combo,"f3")
    kernel = np.ones((5,5),np.uint8)
    dilation = cv2.dilate(np.uint8(combo.astype(np.float32)),kernel,iterations = 1)
    median_image = cv2.medianBlur(np.uint8(dilation.astype(np.float32)),35)
    dilation = cv2.dilate(np.uint8(median_image.astype(np.float32)),kernel,iterations = 3)
    imcon2=np.where(dilation==255,0,255)
    rgb2=cv2.cvtColor(dilation,cv2.COLOR_GRAY2RGB)
    crop_image=np.where(rgb2==(255,255,255),crop_image*0,crop_image)

   
    crop_image=np.where(masked==(0,0,0),crop_image*0,crop_image*1)
      
    return crop_image

In [15]:
# myclip = VideoFileClip('sample_output.mp4')
# output_vid = 'output4.mp4'
# clip = myclip.fl_image(vid_pipeline)
# clip.write_videofile(output_vid, audio=True)

In [16]:
save(vid_pipeline(cv2.imread("frame1.png")),"000")

In [17]:

# cap= cv2.VideoCapture('sample_input.mp4')
# i=0
# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret == False:
#         break
#     cv2.imwrite('f'+str(i)+'.jpg',vid_pipeline(frame))
#     print("-",i,"/3598")
#     i+=1
 
# cap.release()
# cv2.destroyAllWindows()

In [18]:
# import cv2
# import numpy as np
# import glob
 
# img_array = []
# for filename in glob.glob('C:/New folder/Images/*.jpg'):
#     img = cv2.imread(filename)
#     height, width, layers = img.shape
#     size = (width,height)
#     img_array.append(img)
 
 
# out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
# for i in range(len(img_array)):
#     out.write(img_array[i])
# out.release()